# Import libraries and prep spotipy

In [1]:
import spotipy
import spotify_cred
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import pprint
from time import sleep
from random import randint


c_id = spotify_cred.c_id
c_se = spotify_cred.c_se

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=c_id, client_secret=c_se))

## Custom functions

In [2]:
seconds_to_wait = randint(2,7)
def get_playlist_tracks(playlist_id):
    results = sp.playlist_tracks(playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
        sleep(seconds_to_wait)
    return tracks


def get_playlist_audio_features(playlist):
    list_of_audio_features=[]
    bad_items=[]
    for item in range(0,len(playlist)):
        try:
            temp_features=sp.audio_features(playlist[item]["track"]["id"])[0]
            if temp_features!= None:
                list_of_audio_features.append(temp_features)
                #print("Good",item)
        except:
            bad_items.append(item)
            print("Bad",item)
            continue
    df=pd.DataFrame(list_of_audio_features)    
    #df=df[["danceability","energy","loudness","speechiness","acousticness","instrumentalness","liveness","valence","tempo","id","duration_ms"]]
    return df,bad_items

def compile_playlists(playlist_df):
    full_playlist=[]
    seconds_to_wait = randint(2,7)


    for i in playlist_df['IDs']:
        temp_playlist=get_playlist_tracks(i)
        full_playlist.extend(temp_playlist)
        sleep(seconds_to_wait)
    return full_playlist

# Get tempo information per sons

In [3]:
# Get DF
art_df=pd.read_csv("Data/Book2.csv")

In [4]:
art_df['id']

0       6n33KqeNMDPdpxtBPy0Dkk
1       4LXLoOWeJyK3Dn0TUhS1rd
2       2WsOOI6Irm4RJn8GpakHAv
3       2N0F739pJcD6b4uQe6NxDr
4       04Q4ravTeX4rdrTg0dZ9I9
                 ...          
5101    034KyQEk7IhbLtniVhBErj
5102    28FwYInMHQYAZCZtX2rcWl
5103    5teOR8KU4XNWY2WlVUDiIR
5104    1FPTtFzvWMtV1A7ouK78at
5105    64z0EqhVCYOCbEbY7GqECc
Name: id, Length: 5106, dtype: object

In [8]:
genre_list=[]
for i in art_df['id'][400:1000]:
    y=sp.track(i)['album']['artists'][0]['id']
    try:
        z=sp.artist(y)['genres'][0]
    except:
        z="none"
    genre_list.append(z)
    sleep(3)
    print(len(genre_list))
    print(z)
genre=pd.DataFrame(genre_list)

genre.to_csv(r"Data\genrefirst1000.csv")

1
norwegian space disco
2
contemporary r&b
3
chill beats
4
atl hip hop
5
none
6
album rock
7
cologne electronic
8
freestyle
9
none
10
none
11
german jazz
12
k-pop
13
alt z
14
french rock-and-roll
15
new jersey underground rap
16
none
17
alternative americana
18
cello
19
soca


PermissionError: [Errno 13] Permission denied: 'Data\\genrefirst1000.csv'

In [7]:
genre=pd.DataFrame(genre_list)
genre.to_csv(r"Data\genrefirst1000.csv")

In [21]:
art_df['genre']=genre_list
art_df.to_csv("Data\genre800012.csv")

# Get genre info per artist

In [ ]:
# get genre
for i in track_df:
    

In [27]:
x=sp.audio_analysis('3AhXZa8sUQht0UEdBJgpGc')
x

In [31]:
x['track']['tempo']

95.263

In [45]:
y=sp.track('3AhXZa8sUQht0UEdBJgpGc')['album']['artists'][0]['id']
y

'74ASZWbe4lXaubB36ztrGX'

In [46]:
z=sp.artist(y)


In [49]:
z['genres'][0]

'classic rock'

# Get list of songs from playlists

In [5]:
playlist_df=pd.read_csv(r"C:\Users\MichaelTaylo_c9zoof3\Documents\GitHub\Ironhack-Labs\Spotify\Data\Playlists to parse\10000 song playlist.csv")

In [6]:
song_list=compile_playlists(playlist_df)

In [7]:
len(song_list)

10000

## Save list of songs for future use

In [8]:
import pickle
with open("Data\song_list.pickle", "wb") as f:
	pickle.dump(song_list,f)

## Compile list of audio features from song list

In [9]:
df,bad_list=get_playlist_audio_features(song_list)

Bad 467
Bad 2658
Bad 2671
Bad 2979
Bad 3387
Bad 3470
Bad 3640
Bad 3684
Bad 3731
Bad 3757
Bad 3775
Bad 4003
Bad 4049
Bad 4318
Bad 4465
Bad 4468
Bad 4552
Bad 4710
Bad 4905
Bad 5459
Bad 5526
Bad 5926
Bad 6094
Bad 6179
Bad 6243
Bad 6663
Bad 6846
Bad 6897
Bad 6908
Bad 7131
Bad 7173
Bad 7336
Bad 7579
Bad 7655
Bad 7693
Bad 7786
Bad 7793
Bad 7909
Bad 8464
Bad 8475
Bad 8660
Bad 8828
Bad 8911
Bad 8941
Bad 9162
Bad 9382
Bad 9489
Bad 9633
Bad 9693
Bad 9819
Bad 9846
Bad 9850
Bad 9865


## Save dataframe of audio features for future use

In [10]:
df.to_csv("Data\compiled10000.csv", index=False)

'Bob Dylan'

# Add on additional information

In [21]:
artists=[]
song_titles=[]
popularity=[]
explicit=[]
release_date=[]

for ids in df['id']:
    temp_val=sp.track(ids)
    song_titles.append(temp_val['name'])
    popularity.append(temp_val['popularity'])
    release_date.append(temp_val['album']['release_date'])
    explicit.append(temp_val['explicit'])
    artists.append(temp_val['artists'][0]['name'])

In [22]:
df_full=df.copy()

In [23]:
df_full['artist']=artists
df_full['song_title']=song_titles
df_full['explicit']=explicit
df_full['popularity']=popularity

# Save final version of file

In [24]:
df_full.to_csv("Data\compiled_full_10000.csv", index=False)